# MNAD Evaluation

## Libraries import

In [10]:
import argparse
import os
import torch
import torch.utils.data as data
import torchvision.transforms as T
import torch.nn as nn

from data.CustomDataset import CustomImageDataset
from evaluation_utils import *

## Constants

In [11]:
DATASET_DIR_SUFFIX = 'images'

## Parameters

In [12]:
args_dict = {
  "gpus": "1",                            # gpus (set 1 or None)
  "batch_size": 1,                        # batch size for testing
  "h": 256,                               # height of input images
  "w": 256,                               # width of input images
  "c": 3,                                 # channel of input images
  "method": "recon",                      # The target task for anoamly detection  (pred or recon)
  "t_length": 1,                          # length of the frame sequences
  "fdim": 512,                            # channel dimension of the features
  "mdim": 512,                            # channel dimension of the memory items
  "msize": 10,                            # number of the memory items
  "alpha": 0.7,                           # weight for the anomality score
  "th": 0.015,                            # threshold for test updating
  "num_workers": 1,                       # number of workers for the test loader
  "dataset_type": "clean_road",           # type of dataset: clean_road
  "dataset_path": "./dataset",            # directory of data
  "label_path": "./dataset",              # directory of labels
  "label_file": "test_labels.csv",        # name of the label file
  "model_path": "model/trained",          # directory of model
  "model_file": "model.pth",              # name of the model file
  "m_items_path": "model/trained",        # directory of memory items
  "m_items_file": "keys.pt"               # name of the memory items file
}

args = argparse.Namespace(**args_dict)

## GPU Configurations

In [13]:
print(torch.cuda.is_available())
if args.gpus is not None and torch.cuda.is_available():
  print(torch.cuda.is_available())

  print(torch.cuda.device_count())

  print(torch.cuda.current_device())

  print(torch.cuda.device(0))

  print(torch.cuda.get_device_name(0))

True
True
1
0
Quadro T2000 with Max-Q Design


In [14]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
if args.gpus is None:
    gpus = "0"
    os.environ["CUDA_VISIBLE_DEVICES"]= gpus
else:
    gpus = ""
    for i in range(len(args.gpus)):
        gpus = gpus + args.gpus[i] + ","
    os.environ["CUDA_VISIBLE_DEVICES"]= gpus[:-1]

#torch.backends.cudnn.enabled = True # make sure to use cudnn for computational performance

## Data loading

In [15]:
test_folder = os.path.join(args.dataset_path, args.dataset_type, DATASET_DIR_SUFFIX)
test_label_file = os.path.join(args.label_path, args.dataset_type, args.label_file)

#transform = T.Resize((args.h,args.w))
transform = T.Compose([T.ToTensor(),])

# Loading dataset
test_dataset = CustomImageDataset(test_label_file, test_folder, transform = transform, use_cv2=True)
test_size = len(test_dataset)

test_batch = data.DataLoader(test_dataset, batch_size = args.batch_size,
                              shuffle=True, num_workers=args.num_workers, drop_last=False)
batch_size = len(test_batch)

## Model loading

In [16]:
# Loading the trained model
model_file = os.path.join(args.model_path, args.dataset_type, args.model_file)
m_items_file = os.path.join(args.m_items_path, args.dataset_type, args.m_items_file)
#model = torch.load(args.model_dir, map_location=torch.device('cpu'))
model = torch.load(model_file)
model.cuda()
#m_items = torch.load(args.m_items_dir, map_location=torch.device('cpu'))
m_items = torch.load(m_items_file)

## Evaluation

In [17]:
loss_func_mse = nn.MSELoss(reduction='none')

psnr_list = {}
feature_distance_list = {}

# Populate the dictionaries with the image names and empty lists
for img_name in test_dataset.imgs_labels["filename"].to_numpy():
    psnr_list[img_name] = []
    feature_distance_list[img_name] = []

m_items_test = m_items.clone()

In [18]:
print('Evaluation of', args.dataset_type)
model.eval()

for j,(images, labels) in enumerate(test_batch):
    imgs = images["file"]
    img_name = images["name"][0]

    if args.gpus is not None and torch.cuda.is_available():
        imgs = imgs.cuda()

    if args.method == 'pred':
        outputs, feas, updated_feas, m_items_test, softmax_score_query, softmax_score_memory, _, _, _, compactness_loss = model.forward(imgs[:,0:3*4], m_items_test, False)
        mse_imgs = torch.mean(loss_func_mse((outputs[0]+1)/2, (imgs[0,3*4:]+1)/2)).item()
        mse_feas = compactness_loss.item()

        # Calculating the threshold for updating at the test time
        point_sc = point_score(outputs, imgs[:,3*4:])

    else:
        outputs, feas, updated_feas, m_items_test, softmax_score_query, softmax_score_memory, compactness_loss = model.forward(imgs, m_items_test, False)
        mse_imgs = torch.mean(loss_func_mse((outputs[0]+1)/2, (imgs[0]+1)/2)).item()
        mse_feas = compactness_loss.item()

        # Calculating the threshold for updating at the test time
        point_sc = point_score(outputs, imgs)

    if  point_sc < args.th:
        query = nn.functional.normalize(feas, dim=1)
        query = query.permute(0,2,3,1) # b X h X w X d
        m_items_test = model.memory.update(query, m_items_test, False)
    
    psnr_list[img_name].append(psnr(mse_imgs))
    feature_distance_list[img_name].append(mse_feas)

# Measuring the abnormality score and the AUC
anomaly_score_total_list = []

# Calculating the abnormality score as the sum of the PSNR (inverted) and the feature distance
psnr_listed = anomaly_score_list_inv(list(psnr_list.values()))
feature_distance_listed = anomaly_score_list(list(feature_distance_list.values()))
anomaly_score_total_list = score_sum(psnr_listed, feature_distance_listed, args.alpha)

anomaly_score_total_list = np.asarray(anomaly_score_total_list)

print(anomaly_score_total_list.shape)
print(anomaly_score_total_list)

# Calculating the AUC
anomaly_score_total_list = np.expand_dims(anomaly_score_total_list, axis=0)
labels_list = np.expand_dims(test_dataset.imgs_labels["label"].to_numpy(), axis=0)

accuracy = AUC(anomaly_score_total_list, labels_list)

print('The result of ', args.dataset_type)
print('AUC: ', accuracy*100, '%')

Evaluation of clean_road
(73, 1)
[[0.59364452]
 [0.44457223]
 [0.4220863 ]
 [0.52773352]
 [0.95523927]
 [0.46455824]
 [0.42038965]
 [0.57264705]
 [0.47618166]
 [0.59732785]
 [0.75484728]
 [0.39129666]
 [0.64701694]
 [0.75566816]
 [0.60088984]
 [0.23778584]
 [0.46126253]
 [0.74458321]
 [0.64864812]
 [0.37716803]
 [0.66398272]
 [0.58549444]
 [0.51925204]
 [0.85933824]
 [0.37157685]
 [0.30602963]
 [0.80176805]
 [0.55424223]
 [0.93456588]
 [0.        ]
 [0.49495765]
 [0.54696636]
 [0.54864325]
 [0.28679452]
 [0.28982122]
 [0.56100851]
 [0.83177449]
 [0.77087208]
 [0.60992194]
 [0.65712221]
 [0.31816818]
 [0.19919601]
 [0.56955331]
 [0.6450349 ]
 [0.95300313]
 [0.28992424]
 [0.56960272]
 [0.47048443]
 [0.5396862 ]
 [0.4732131 ]
 [0.75659953]
 [0.61850229]
 [0.51510994]
 [0.14154888]
 [0.69708666]
 [0.59990983]
 [0.38340262]
 [0.66253735]
 [0.56972592]
 [0.78451737]
 [0.53502412]
 [0.65140587]
 [0.05301594]
 [0.31847112]
 [0.52628148]
 [0.49007027]
 [0.42006926]
 [0.4398455 ]
 [0.5250926 ]
 